In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
from pathlib import Path

data_path = Path("data/case1")

In [2]:
struct_h = mp.Struct.from_frame(mp.io.read_pdb(data_path / "H.pdb"), "H")
struct_n = mp.Struct.from_frame(mp.io.read_pdb(data_path / "N.pdb"), "N")
struct_m = mp.Struct.from_frame(mp.io.read_pdb(data_path / "M.pdb"), "M")
struct_p = mp.Struct.from_frame(mp.io.read_pdb(data_path / "P.pdb"), "P")
struct_t = mp.Struct.from_frame(mp.io.read_pdb(data_path / "T.pdb"), "T")

typifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
typifier.typify(data_path, struct_h)
typifier.typify(data_path, struct_n, net_charge=-1)
typifier.typify(data_path, struct_m, net_charge=1)
typifier.typify(data_path, struct_p)
typifier.typify(data_path, struct_t)

<Struct: 22 atoms>

In [16]:
reacter = mp.reacter.AmberToolsReacter()
reacter.react(
    data_path / "H",
    struct_h,
    tail=struct_h.atoms["C8"],
    omits=struct_h.atoms[["H6"]],
)
reacter.react(data_path / "T", struct_t, head=struct_t.atoms["C"], omits=struct_t.atoms[["H4"]])


TAIL_ATOM     15   C8
MAIN_CHAIN     1   15   C8
OMIT_ATOM      1   16   H6
Number of mainchain atoms (including head and tail atom):     1
Number of omited atoms:     1
HEAD_ATOM      1    C
MAIN_CHAIN     1    1    C
OMIT_ATOM      1    9   H4
Number of mainchain atoms (including head and tail atom):     1
Number of omited atoms:     1

<Struct: 22 atoms>

In [17]:
polymerizer = mp.polymerizer.AmberToolsPolymerizer()
bMIL3PEGM3 = polymerizer.linear(
    "HT",
    "HT",
    {"H": struct_h, "N": struct_n, "M": struct_m, "P": struct_p, "T": struct_t},
    data_path,
)

T
H
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/atomic_ions.lib
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/solvents.lib
Loa

In [26]:
tfsi = mp.Struct.from_frame(mp.io.read_pdb(data_path / "tfsi.pdb"), "tfsi")
tfsi = typifier.typify(data_path, tfsi, net_charge=-1, is_frcmod=True)
litfsi = polymerizer.unit("litfsi", tfsi, data_path, ion="LI")

poly_ff = mp.io.read_amber(data_path/"HT/HT.prmtop", data_path/"HT/HT.inpcrd")
litfsi_ff = mp.io.read_amber(data_path/"litfsi/litfsi.prmtop", data_path/"litfsi/litfsi.inpcrd")

-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/atomic_ions.lib
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/solvents.lib
Loading

In [ ]:
n_polymer_atoms = len(bMIL3PEGM3.atoms)
n_litfsi_atoms = len(litfsi.atoms)
n_atoms = 20 * n_polymer_atoms + 108 * n_litfsi_atoms
density = 0.1

packer = mp.packer.Packmol(data_path/"bMIL3PEGM3_n20_litfsi_n108")
box = mp.packer.InsideBoxConstraint(
    length=(n_atoms/density) ** (1/3),
)
packer.def_target(
    bMIL3PEGM3, number=20, constraint=box
)
packer.def_target(
    litfsi, number=108, constraint=box,
)
frame = packer.pack()
# frame.forcefield = mp.forcefield.merge(
#     bMIL3PEGM3.forcefield, litfsi_ff.forcefield
# )

# mp.io.write_lammps(
#     data_path / "bMIL3PEGM3_n20_litfsi_n108"),
#     frame
# )  # data file and forcefield